In [303]:
import pandas as pd
import numpy as np
import json
import re
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D  # for 3D plotting
import plotly.express as px
from sklearn.manifold import TSNE
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import plotly.io as pio
import requests
import pandas as pd
import aiohttp
import asyncio
import nest_asyncio
import os
from os import getenv
from dotenv import load_dotenv
import time
import ast 

load_dotenv("../.env",override=True)
GEO_KEY = getenv("GOOGLE_API_KEY")

x_chat = chatGPT()
x_gemini = gemini()


In [304]:
df = pd.read_csv("../output/results/bls_df.csv",index_col=0)
tsne = pd.read_csv("../output/results/tsne.csv",index_col=0)
df = pd.concat([df,tsne],axis=1)

# df.example_task_embedding = df.example_task_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
# df.onet_task_embedding = df.onet_task_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])
# df.onet_title_embedding = df.onet_title_embedding.apply(lambda x: [float(y) for y in x.strip("[]").split(", ")])


In [305]:
df2 = df

In [379]:
df = df2
df.head()

,organization name,number,Example,example_task_embedding,founded date,website,description_all,industries_parsed,generated_description,parsed_description,...,Task,onet_task_embedding,onet_title_embedding,Task ID,example_task_embedding_tsne1,example_task_embedding_tsne2,onet_task_embedding_tsne1,onet_task_embedding_tsne2,onet_title_embedding_tsne1,onet_title_embedding_tsne2
0,Pika,1,A content creator uses Pika to quickly turn a ...,"[-0.009328115731477737, -0.004033779725432396,...",2023-01-01,pika.art,Pika is a startup that develops an AI-powered ...,"['Generative AI', 'Graphic Design', 'Video']",Pika.art is an AI-powered platform that allows...,"Tasks/Jobs: Video editing, Filmmaking, Storybo...",...,Trim film segments to specified lengths and re...,"[-0.014954066835343838, 0.007443919777870178, ...","[0.007084680255502462, -0.0037158424966037273,...",4066,20.661396,7.479743,-20.600520,-12.787438,21.840174,-15.022711
1,Contextual AI,1,A marketing team uses Contextual AI to brainst...,"[-0.0026594670489430428, -0.01533527858555317,...",2023-01-01,contextual.ai,Contextual AI offers a pioneering approach to ...,"['Generative AI', 'Software']",Contextual AI is a company that provides a gen...,"Tasks/Jobs: Brainstorming, Content creation, K...",...,"Formulate, direct, or coordinate marketing act...","[-0.021589307114481926, -0.014030396938323975,...","[-0.008224736899137497, -0.008055020123720169,...",20709,5.094541,-11.276998,-16.065542,-10.083408,-22.837185,11.307454
2,Contextual AI,1,A marketing team uses Contextual AI to brainst...,"[-0.0026594670489430428, -0.01533527858555317,...",2023-01-01,contextual.ai,Contextual AI offers a pioneering approach to ...,"['Generative AI', 'Software']",Contextual AI is a company that provides a gen...,"Tasks/Jobs: Brainstorming, Content creation, K...",...,Develop or implement product-marketing strateg...,"[-0.01351824589073658, -0.023576783016324043, ...","[0.011500373482704163, -0.012599950656294823, ...",945,5.094541,-11.276998,-17.554142,-5.238864,-6.627986,1.440020
3,Contextual AI,1,A marketing team uses Contextual AI to brainst...,"[-0.0026594670489430428, -0.01533527858555317,...",2023-01-01,contextual.ai,Contextual AI offers a pioneering approach to ...,"['Generative AI', 'Software']",Contextual AI is a company that provides a gen...,"Tasks/Jobs: Brainstorming, Content creation, K...",...,"Coordinate with marketing team members, graphi...","[-0.02083304338157177, -0.022601015865802765, ...","[0.007197009399533272, -0.01158437505364418, -...",21221,5.094541,-11.276998,-15.637196,-8.300720,-19.454912,12.127976
4,Sierra,1,A customer service representative uses a conve...,"[-0.02811567112803459, -0.030599573627114296, ...",2023-01-01,sierra.ai,Sierra is an AI startup that tackles essential...,"['Enterprise Software', 'SaaS']",Sierra is a company that provides AI-powered c...,"Tasks/Jobs: Customer support, Lead generation,...",...,Confer with customers by telephone or in perso...,"[-0.021748239174485207, -0.024130303412675858,...","[0.0009099573944695294, -0.0367814339697361, -...",18565,-4.505715,36.209260,-3.150575,23.005049,0.288652,-19.666649


In [380]:
onet_occ = pd.read_csv("../input/onet/Occupation Data.csv")[["O*NET-SOC Code","Title"]]
onet_occ.columns = ["Detailed Occupation","onet_title"]
onet_occ["Detailed Occupation"] = onet_occ["Detailed Occupation"].apply(lambda x: x[:-3])


In [381]:
wage_employment = pd.read_csv("../input/wage_employment_2022/national2022.csv")
# wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
wage_employment = wage_employment.replace("#", np.nan).replace("*", np.nan).replace({",": ""}, regex=True)
wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP"]].astype("float")
wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
wage_employment = wage_employment[["Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP"]]
wage_employment.columns = ["Detailed Occupation","H_MEDIAN_US","A_MEDIAN_US","TOT_EMP_US"]
wage_employment = wage_employment.groupby('Detailed Occupation').mean().reset_index()


In [382]:
onet_occ = onet_occ.merge(wage_employment[["Detailed Occupation","TOT_EMP_US"]], on="Detailed Occupation", how="left")
null_bls = pd.DataFrame([[x,x[:-1]+"0"] for x in list(onet_occ[onet_occ["TOT_EMP_US"].isnull()]["Detailed Occupation"].unique())],columns=["Detailed Occupation","BLS Detailed Occupation"])
bls_occ = onet_occ[["Detailed Occupation"]].merge(null_bls, on="Detailed Occupation", how="left")[["Detailed Occupation", "BLS Detailed Occupation"]]
bls_occ['BLS Detailed Occupation'] = bls_occ['BLS Detailed Occupation'].fillna(bls_occ['Detailed Occupation'])
onet_occ = onet_occ.drop(columns=["TOT_EMP_US"])
bls_occ.groupby("Detailed Occupation")["BLS Detailed Occupation"].first().reset_index()

wage_employment = wage_employment.rename({"Detailed Occupation":"BLS Detailed Occupation"},axis=1)
wage_employment = wage_employment.merge(bls_occ.groupby("Detailed Occupation")["BLS Detailed Occupation"].first().reset_index(), on="BLS Detailed Occupation")
wage_employment = wage_employment.drop(columns=["BLS Detailed Occupation"]).rename({"BLS Detailed Occupation":"Detailed Occupation"},axis=1)


In [387]:
all_tasks = df.groupby(["Task","Title"]).aggregate({"onet_weight":"sum"}).reset_index()
task_statements = pd.read_csv("../input/onet/Task Statements.csv")[["Task","Title"]]
all_tasks = all_tasks.merge(task_statements, on=["Task","Title"], how="outer")
all_tasks = all_tasks.merge(onet_occ, left_on="Title",right_on="onet_title").drop(columns=["onet_title"])

all_tasks = all_tasks.replace(np.nan,0)
perc_of_workers = (wage_employment.groupby("Detailed Occupation")["TOT_EMP_US"].first()/wage_employment["TOT_EMP_US"].sum()).reset_index() # num of people for every 10K workers
perc_of_workers = perc_of_workers.rename({"TOT_EMP_US":"Percent of workers"},axis=1)


all_tasks = all_tasks.merge(perc_of_workers, on="Detailed Occupation")
ratios = all_tasks['onet_weight'] / all_tasks['Percent of workers']
all_tasks["task_automation_rating"] = np.minimum(ratios, 1)


all_occupations = all_tasks.groupby("Detailed Occupation").aggregate({"task_automation_rating":"sum","Task":"count", "Title":"first"})
all_occupations.columns = ["occupation_# tasks automated","occupation_# of tasks","Title"]
all_occupations["occupation_automation_rating"] = all_occupations["occupation_# tasks automated"]/all_occupations["occupation_# of tasks"]
df = df.merge(all_tasks[["Task","Title","task_automation_rating","Percent of workers"]], on=["Task","Title"], how="left")
df = df.merge(all_occupations,on=["Detailed Occupation","Title"],how="left")

df = df.merge(wage_employment, on="Detailed Occupation", how="left")
all_occupations = all_occupations.merge(wage_employment, on="Detailed Occupation")
all_tasks = all_tasks.merge(wage_employment, on="Detailed Occupation")


In [388]:
codes = pd.read_csv("../input/soc_codes/soc_codes.csv", index_col=0).drop(columns=["Title"])
all_occupations = all_occupations.merge(codes,on="Detailed Occupation", how="left")
all_tasks = all_tasks.merge(codes,on="Detailed Occupation", how="left")


In [389]:
job_zones = pd.read_csv("../input/onet/Job Zones.csv",index_col=0).reset_index()[["O*NET-SOC Code","Job Zone"]]
job_zones.columns = ["Detailed Occupation","Job Zone"]
job_zones["Detailed Occupation"] = job_zones["Detailed Occupation"].apply(lambda x: x[:-3])
job_zones = job_zones.groupby(["Detailed Occupation"])["Job Zone"].mean().reset_index()
job_zones_ref = pd.read_csv("../input/onet/Job Zone Reference.csv",index_col=0)
df = df.merge(job_zones, on="Detailed Occupation", how="left")
all_occupations = all_occupations.merge(job_zones, on="Detailed Occupation", how="left")
all_tasks = all_tasks.merge(job_zones, on="Detailed Occupation", how="left")
len(df)

8120

In [391]:
education = pd.read_csv("../input/onet/Education, Training, and Experience.csv",index_col=0).reset_index()[["O*NET-SOC Code","Data Value","Category", "Scale Name"]]
education.columns = ["Detailed Occupation","Data Value","Category", "Scale Name"]
education["Detailed Occupation"] = education["Detailed Occupation"].apply(lambda x: x[:-3])
total = education.groupby(["Detailed Occupation","Scale Name"])["Data Value"].sum().reset_index().rename({"Data Value":"Total"},axis=1)
education = education.merge(total,on=["Detailed Occupation","Scale Name"])
education["weighted_category"] = education["Data Value"]/education["Total"]*education["Category"]
education = education.groupby(["Detailed Occupation","Scale Name"])["weighted_category"].sum().reset_index()
education = education.pivot(index=['Detailed Occupation'], columns='Scale Name', values='weighted_category').reset_index().drop(columns=["Importance"])
education.columns.name = None
education.columns = [col if not isinstance(col, tuple) else col[1] for col in education.columns]
df = df.merge(education, on="Detailed Occupation", how="left")
all_occupations = all_occupations.merge(education, on="Detailed Occupation", how="left")
all_tasks = all_tasks.merge(education, on="Detailed Occupation", how="left")
len(df)

8120

In [393]:
def parse_bls(wage_employment,type):
    wage_employment = wage_employment[wage_employment.O_GROUP == "detailed"]
    wage_employment = wage_employment.rename({"OCC_CODE": "Detailed Occupation"}, axis=1)
    wage_employment = wage_employment.replace("#", np.nan).replace(r"\*+", np.nan, regex=True).replace(",", "", regex=True)
    wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP","JOBS_1000"]] = wage_employment[["H_MEDIAN", "A_MEDIAN","TOT_EMP", "JOBS_1000"]].astype("float")
    wage_employment["H_MEDIAN"] = wage_employment["H_MEDIAN"].round(0)
    wage_employment = wage_employment[["AREA","Detailed Occupation","H_MEDIAN","A_MEDIAN","TOT_EMP","JOBS_1000"]]
    wage_employment.columns = ["AREA","Detailed Occupation","H_MEDIAN_"+type,"A_MEDIAN_"+type,"TOT_EMP_"+type, 'JOBS_1000_'+type]
    return wage_employment


nonmetro = pd.read_csv("../input/wage_employment_2022/rural2022.csv")
metro = pd.read_csv("../input/wage_employment_2022/metro2022.csv")
nonmetro = parse_bls(nonmetro,"RURAL")
metro = parse_bls(metro,"METRO")

#figures out how much to weight each area
def calculate_weights(df,type):
    grouped = df.groupby("Detailed Occupation")["TOT_EMP_"+type].sum().reset_index()
    grouped = grouped.rename({"TOT_EMP_"+type: "sum_"+type}, axis=1)
    df = df.merge(grouped, on="Detailed Occupation")
    df["weight_area_"+type] = df["TOT_EMP_"+type] / df["sum_"+type]
    df["weighted_JOBS_1000_"+type] = df["JOBS_1000_"+type]*df["weight_area_"+type]
    return df
metro = calculate_weights(metro,"METRO")
nonmetro = calculate_weights(nonmetro,"RURAL")

metro_grouped = metro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_METRO":"mean","TOT_EMP_METRO":"sum","A_MEDIAN_METRO":"mean", "weighted_JOBS_1000_METRO":"sum"},axis=1).reset_index()
nonmetro_grouped = nonmetro.groupby("Detailed Occupation").aggregate({"H_MEDIAN_RURAL":"mean","TOT_EMP_RURAL":"sum","A_MEDIAN_RURAL":"mean","weighted_JOBS_1000_RURAL":"sum"},axis=1).reset_index()

df = df.merge(metro_grouped,on="Detailed Occupation",how="left")
df = df.merge(nonmetro_grouped,on="Detailed Occupation",how="left")

In [394]:
metro.columns = [col.replace("_METRO","") for col in metro.columns]
metro["Type"] = "METRO"
nonmetro.columns = [col.replace("_RURAL","") for col in nonmetro.columns]
nonmetro["Type"] = "RURAL"
locations = pd.concat([metro, nonmetro],axis=0)
locations = locations.merge(all_occupations,on="Detailed Occupation")
locations["weighted_automation_#_per_1000_people"] = locations["JOBS_1000"]*locations["occupation_automation_rating"]
locations = locations.groupby("AREA").aggregate({"weighted_automation_#_per_1000_people":"sum", "Type":"first"}).reset_index()


geo = pd.read_csv("../output/parsed_BLS_data/geography.csv",index_col=0)
geo.Boundary = geo.Boundary.apply(lambda x: ast.literal_eval(x) if not pd.isnull(x) else x)
geo = geo[~geo["name"].str.contains("Census Area")]


locations = locations.merge(geo, on="AREA",how="left")
locations = locations.dropna(subset=["name"])

locations.to_csv("../output/EDA_data/automation_by_geography.csv")

In [395]:
onet_occ = pd.read_csv('../output/all_task_occ_embeddings/occupation_embeddings.csv')
onet_occ = onet_occ[["Detailed Occupation","all_onet_occupations_embedding_tsne1","all_onet_occupations_embedding_tsne2"]]
all_occupations = all_occupations.merge(onet_occ, on=["Detailed Occupation"],how="left")

In [403]:
task_statements = pd.read_csv('../output/all_task_occ_embeddings/task_statement_embeddings.csv')
task_statements = task_statements[["Task","Title","all_onet_tasks_embedding_tsne1","all_onet_tasks_embedding_tsne2"]]
all_tasks = all_tasks.merge(task_statements,on=["Task","Title"],how="left")

In [405]:
all_occupations.to_csv("../output/EDA_data/all_occupations_automation.csv")
all_tasks.to_csv("../output/EDA_data/all_tasks_automation.csv")
df.to_csv("../output/EDA_data/startup_data.csv")

In [318]:
#all_occupations is smaller than onet occ because from task statements which doesn't have broad occuaptions

In [397]:
len(all_tasks)

19371